## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
  import pydrake
  import underactuated
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

# Setup matplotlib backend (to notebook, if possible, or inline).  
from underactuated.jupyter import setup_matplotlib_backend
plt_is_interactive = setup_matplotlib_backend()

# Trajectory optimization for the double integrator

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import eq, MathematicalProgram, Solve, Variable

# Discrete-time approximation of the double integrator.
dt = 0.01;
A = np.eye(2) + dt*np.mat('0 1; 0 0')
B = dt*np.mat('0; 1')

Q = np.eye(2)
R = np.eye(1)
prog = MathematicalProgram()

N = 284  # Note: I had to do a manual "line search" to find this.

# Create decision variables
u = np.empty((1, N-1), dtype=Variable)
x = np.empty((2, N), dtype=Variable)
for n in range(N-1):
  u[:,n] = prog.NewContinuousVariables(1, 'u' + str(n))
  x[:,n] = prog.NewContinuousVariables(2, 'x' + str(n))
x[:,N-1] = prog.NewContinuousVariables(2, 'x' + str(N))

# Add costs and constraints
x0 = [-2, 0]
prog.AddBoundingBoxConstraint(x0, x0, x[:,0])
for n in range(N-1):
  # Will eventually be prog.AddConstraint(x[:,[n+1]] == A*x[:,[n]] + B*u[:,n])
  # See drake issues 12841 and 8315
  dynamics_constraint = eq(x[:,[n+1]],A*x[:,[n]] + B*u[:,n])
  prog.AddConstraint(dynamics_constraint[0, 0])
  prog.AddConstraint(dynamics_constraint[1, 0])
  prog.AddBoundingBoxConstraint(-1, 1, u[:,n])
xf = [0, 0]
prog.AddBoundingBoxConstraint(xf, xf, x[:, N-1])

result = Solve(prog)

x_sol = result.GetSolution(x)
assert(result.is_success()), "Optimization failed"

plt.figure()
plt.plot(x_sol[0,:], x_sol[1,:])
plt.xlabel('q')
plt.ylabel('qdot');

In [ ]:
from pydrake.all import LinearSystem, DirectTranscription

C = np.eye(2)
D = np.zeros((2,1))
sys = LinearSystem(A, B, C, D, dt)

prog = DirectTranscription(sys, sys.CreateDefaultContext(), N)
prog.AddBoundingBoxConstraint(x0, x0, prog.initial_state())
prog.AddBoundingBoxConstraint(xf, xf, prog.final_state())
prog.AddConstraintToAllKnotPoints(prog.input()[0] <= 1)
prog.AddConstraintToAllKnotPoints(prog.input()[0] >= -1)

result = Solve(prog)
x_sol = prog.ReconstructStateTrajectory(result)
assert(result.is_success()), "Optimization failed"

plt.figure()
x_values = x_sol.vector_values(x_sol.get_segment_times())

plt.plot(x_values[0,:], x_values[1,:])
plt.xlabel('q')
plt.ylabel('qdot');